> To live! like a tree alone and free,
> and like a forest in solidarity...
> Nazim Hikmet

# Random Forests



__Objectives__

- Introduction of 'bagging' procedure.

- Identifying the need for bootstrapping for random forests

- Comparing Random forests and bagging methods

- Evaluating a model by random forest model

## Bootstrapping


<img src= "img/bootstrap1.png" style="height:250px">


# Bagging (Boostrapping + Aggregating)


Let's us one more time recall that if $Z_{1}, \cdots, Z_{n}$ are independent observations with variance $\sigma^{2}$ then the variance of the mean $\bar{Z}$ is given by $\frac{\sigma^{2}}{n}$. 

__How is this relevant now?__



We will use this idea calculate $$ \hat{f}^{1}(x), \cdots, \hat{f}^{B}(x)$$ where each $\hat{f}^{i}$ represents a decision tree fitted to the bootstrapped data.

Then we will make a prediction by: 

$$ \hat{f}_{\text{avg}}(x) = \frac{1}{B}\sum_{b=1}^{B} \hat{f}^{b}(x)$$

Note that this is for regression and for the classification we can get majority vote.

_side note: [sklearn averages over probabilities not majority vote](https://scikit-learn.org/stable/modules/ensemble.html#forest)_


## Sklearn for Random Forests

In [85]:
import pandas as pd
import numpy as np

In [86]:
# you can download the data from -- https://www.kaggle.com/ishaanv/ISLR-Auto#Heart.csv

# or http://faculty.marshall.usc.edu/gareth-james/ISL/data.html
heart = pd.read_csv('data/Heart.csv', index_col=0)
heart.head()
print(heart.shape)

(303, 14)


In [87]:
# drop nulls
heart.dropna(axis=0, how='any', inplace=True)
y = heart.AHD
X = heart.drop(columns='AHD')

In [88]:
heart.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [89]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 1 to 302
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        297 non-null    int64  
 1   Sex        297 non-null    int64  
 2   ChestPain  297 non-null    object 
 3   RestBP     297 non-null    int64  
 4   Chol       297 non-null    int64  
 5   Fbs        297 non-null    int64  
 6   RestECG    297 non-null    int64  
 7   MaxHR      297 non-null    int64  
 8   ExAng      297 non-null    int64  
 9   Oldpeak    297 non-null    float64
 10  Slope      297 non-null    int64  
 11  Ca         297 non-null    float64
 12  Thal       297 non-null    object 
 13  AHD        297 non-null    object 
dtypes: float64(2), int64(9), object(3)
memory usage: 34.8+ KB


In [91]:
display(X.Thal.value_counts(normalize=True))
display(X.ChestPain.value_counts(normalize=True))

normal        0.552189
reversable    0.387205
fixed         0.060606
Name: Thal, dtype: float64

asymptomatic    0.478114
nonanginal      0.279461
nontypical      0.164983
typical         0.077441
Name: ChestPain, dtype: float64

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, stratify=y)

In [93]:
from sklearn.preprocessing import OneHotEncoder

In [94]:
# shortcut for splitting catvar from numvar:
categorical_variables = list(X_train.select_dtypes(include=['object']).columns)
numerical_variables = list(X_train.select_dtypes(include=['int64', 'float64']).columns)

In [95]:
categorical_variables

['ChestPain', 'Thal']

In [96]:
numerical_variables

['Age',
 'Sex',
 'RestBP',
 'Chol',
 'Fbs',
 'RestECG',
 'MaxHR',
 'ExAng',
 'Oldpeak',
 'Slope',
 'Ca']

In [97]:
categorical_variables.append(numerical_variables.pop(5))

In [98]:
categorical_variables.append(numerical_variables.pop(-2))

In [99]:
categorical_variables

['ChestPain', 'Thal', 'RestECG', 'Slope']

In [100]:
ohe = OneHotEncoder(drop='first')
X_categ = ohe.fit_transform(X_train[categorical_variables]).toarray()
X_num = X_train[numerical_variables].values
Xtrain = np.concatenate((X_categ, X_num), axis=-1,)
Xtrain.shape

(237, 18)

In [101]:
# check out feature names for cat and num:
ohe.get_feature_names()

array(['x0_nonanginal', 'x0_nontypical', 'x0_typical', 'x1_normal',
       'x1_reversable', 'x2_1', 'x2_2', 'x3_2', 'x3_3'], dtype=object)

In [102]:
numerical_variables

['Age', 'Sex', 'RestBP', 'Chol', 'Fbs', 'MaxHR', 'ExAng', 'Oldpeak', 'Ca']

In [103]:
# now we should transform the test data
# to be able to use it for the prediction

X_test_categ = ohe.transform(X_test[categorical_variables]).toarray()
X_test_num = X_test[numerical_variables].values
Xtest = np.concatenate((X_test_categ, X_test_num), axis=-1,)
Xtest.shape

(60, 18)

In [104]:
Xtest.shape

(60, 18)

In [105]:
from sklearn.ensemble import RandomForestClassifier

In [106]:
clf = RandomForestClassifier(n_estimators=100,
                             criterion='gini',
                             max_features='auto',
                             oob_score=True)

In [107]:
clf.fit(Xtrain, y_train)
print(clf.score(Xtrain, y_train))
print(clf.score(Xtest, y_test))

1.0
0.7666666666666667


__Your Turn__

- Use 5 fold cross_validation to fit random forest classifier we created above.
- Don't forget to return training scores and trained estimators.

In [108]:
from sklearn.model_selection import cross_validate

In [109]:
cv = cross_validate(clf, Xtrain, y_train, return_estimator=True, return_train_score=True)

__Your Turn__

- What is the type of validator above?

- Check test vs train(validation) scores.

- Print "mean +/- std" for both train and test scores

- Also print oob_scores and compare them with cross_validation scores

In [110]:
type(cv)

dict

In [111]:
cv

{'fit_time': array([0.17054105, 0.16129613, 0.19849014, 0.16712976, 0.17132807]),
 'score_time': array([0.00969505, 0.00872397, 0.00849795, 0.01031399, 0.00976586]),
 'estimator': (RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True),
  RandomForestClassifier(oob_score=True)),
 'test_score': array([0.85416667, 0.8125    , 0.74468085, 0.74468085, 0.82978723]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [112]:
clf.fit(Xtrain, y_train)
clf.oob_score_

0.810126582278481

In [114]:
# How to interpret: if one sample has really bad test_score, it may mean there's some really bad data in test.
# I may want to go back and change my model. 
cv['test_score'].mean()

0.7971631205673759

__Your Turn__

- Note that we have over-fitting problem. 

- Let's try to reduce over-fitting

In [115]:
y_train.value_counts()

No     128
Yes    109
Name: AHD, dtype: int64

In [116]:
clf = RandomForestClassifier(max_depth=10,
                             max_features='log2',
                             min_samples_split=4,
                             oob_score=True)

clf.fit(Xtrain, y_train)
print(clf.score(Xtrain, y_train))
print(clf.score(Xtest, y_test))
print(clf.oob_score_)

0.9957805907172996
0.7666666666666667
0.7974683544303798


### Do it with Pipelines!

In [117]:
# There is an "easier" way to do this
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [126]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numerical_variables),
        ('cat', categorical_transformer, categorical_variables)],)

rf = Pipeline(steps=[
    ('ct', preprocessor),
    ('clf', RandomForestClassifier(oob_score=True))])

In [127]:
pipe_validator = cross_validate(rf, X_train, y_train, return_estimator=True, return_train_score=True)

In [128]:
# train scores
print(pipe_validator['train_score'])
# validation scores
print(pipe_validator['test_score'])

[1. 1. 1. 1. 1.]
[0.85416667 0.79166667 0.80851064 0.78723404 0.85106383]


In [146]:
# let's pick one of the estimator for further investigation
est = pipe_validator['estimator'][0]
est

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Sex', 'RestBP',
                                                   'Chol', 'Fbs', 'MaxHR',
                                                   'ExAng', 'Oldpeak', 'Ca']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['ChestPain', 'Thal',
                              

In [148]:
type(est)

sklearn.pipeline.Pipeline

In [149]:
est['clf'].oob_score_

0.8148148148148148

## Feature Importance

In [121]:
feature_importances = clf.feature_importances_

In [122]:
feature_importances

array([0.03847739, 0.0135596 , 0.01827608, 0.11683743, 0.11090284,
       0.00026526, 0.01878719, 0.03254211, 0.0042062 , 0.08449287,
       0.03896566, 0.07388221, 0.07778826, 0.00969994, 0.0925172 ,
       0.06325108, 0.10101562, 0.10453306])

In [123]:
ohe.get_feature_names()

array(['x0_nonanginal', 'x0_nontypical', 'x0_typical', 'x1_normal',
       'x1_reversable', 'x2_1', 'x2_2', 'x3_2', 'x3_3'], dtype=object)

In [124]:
# be careful with the order of columns
columns = ohe.get_feature_names().tolist() + numerical_variables

In [125]:
importances = pd.DataFrame(data=feature_importances,
                           index=columns, columns=['feature_importances'])

importances.sort_values(by='feature_importances', ascending=False)

,feature_importances
x1_normal,0.116837
x1_reversable,0.110903
Ca,0.104533
Oldpeak,0.101016
MaxHR,0.092517
Age,0.084493
Chol,0.077788
RestBP,0.073882
ExAng,0.063251
Sex,0.038966


In [60]:
importances.feature_importances.sum()

1.0000000000000002

In [59]:
# Interpret of feature importance: 
# How much information gain do I get from a split on x1_reversable? 
# each number shows the % of model that's explained by a given variable.

### Extra Material 

- [Sklearn averages probabilities in RF implementation](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [On the variance](https://newonlinecourses.science.psu.edu/stat414/node/167/)

- [Is RF immune to overfitting?](https://en.wikipedia.org/wiki/Talk%3ARandom_forest)

- [Tricky stuff with respect to feature importance](http://rnowling.github.io/machine/learning/2015/08/10/random-forest-bias.html)

- [An interesting implementation of feature importance](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances_faces.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-faces-py)

- [Different Ensemble Methods in sklearn](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [ISLR - section 8.2](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf)

- [Another library for RF: H2o](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html)